In [ ]:
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import pickle

df = pd.read_csv('/content/hate_speech.tsv', sep='\t').fillna(' ') #read_excel('/content/train.xlsx')
print(df.head())
# Create a list of punctuation marks
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '•', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']

# Code to replace punctuations with whitespaces
def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, ' ')
    return x
df['Cleaned'] = df['Text'].apply(lambda x: re.sub(r'http\S+', '', x))
df['Cleaned'] = df['Cleaned'].apply(lambda x: re.sub("@[\w]*", '', x))
df['Cleaned'] = df['Cleaned'].apply(lambda x: clean_text(x))
df['Cleaned'] = df['Cleaned'].str.lower()
df['Cleaned'] = df['Cleaned'].apply(lambda x:' '.join(x.split()))
df['Sentiment'] = df['Label']
df = df.drop(['Text','Label'],axis=1)
print(df.head())
!pip install transformers
import torch
from torch.autograd import Variable
from torch import IntTensor
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = TFBertModel.from_pretrained("bert-base-multilingual-cased")

tokenized = batch['Cleaned'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length=40,truncation=True)))
max_len = 0
for i in tokenized.values:
  #print(len(i))
  if len(i) > max_len:
    max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

print(padded)
output = model(padded)
embedding=output.last_hidden_state
features = embedding[:,0,:].numpy()
bert2=bert[0:6000]
df = pd.read_csv('/content/train (1).csv') 
y = df['label']

y = np.array(list(map(lambda x: 1 if x=="real" else 0, y)))


In [ ]:
from sklearn.model_selection import train_test_split
berttrain_features, berttest_features, berttrain_labels, berttest_labels = train_test_split(features, y,  test_size=0.3,random_state=50)
input_bert = Input(shape=(768,))
dense_text = Dense(1000,activation='relu',kernel_regularizer=regularizers.l2(0.01), kernel_initializer=initializers.he_normal(seed=0))(input_bert)
#dense_text = Dropout(0.4)(dense_text)
dense_text = Dense(500,activation='relu',kernel_regularizer=regularizers.l2(0.01), kernel_initializer=initializers.he_normal(seed=0))(dense_text)
dense_text = Dropout(0.4)(dense_text)
dense_text = Dense(100,activation='relu',kernel_regularizer=regularizers.l2(0.01), kernel_initializer=initializers.he_normal(seed=0))(dense_text)
dense_text = BatchNormalization()(dense_text)
dense_bert_drop = Dropout(0.4)(dense_text)
output = Dense(1, activation='sigmoid')(dense_bert_drop)

model = Model(inputs=[input_bert,input_elmo], outputs=output)
adam = optimizers.Adam(lr=1e-4)
checkpoint = ModelCheckpoint(filepath='../checkpoints_gossip/dense_MM_model.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#sgd = optimizers.SGD(lr=1e-4, clipnorm=1.)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()



In [ ]:
history = model.fit([berttrain_features,elmotrain_features],berttrain_labels,batch_size =32,epochs=100)
score = model.evaluate([berttest_features,elmotest_features] ,berttest_labels, verbose=1)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])
